In [ ]:
import requests
from geopy.distance import geodesic
import json

def get_route(origin, destination, api_key):
    url = f"https://maps.googleapis.com/maps/api/directions/json?origin={origin}&destination={destination}&key={api_key}"
    response = requests.get(url)
    data = response.json()

    if data["status"] == "OK":
        route = data["routes"][0]["legs"][0]
        steps = route["steps"]
        points = [(step["end_location"]["lat"], step["end_location"]["lng"]) for step in steps]
        return points
    else:
        raise Exception("Error getting route from Google Maps API")

def find_charging_stations(lat, lon, api_key):
    url = f"https://api.openchargemap.io/v3/poi/?output=json&latitude={lat}&longitude={lon}&maxresults=10&key={api_key}"
    response = requests.get(url)
    return response.json()

def find_stops(points, max_range, starting_range, buffer_range, charging_api_key):
    stops = []
    distance_traveled = 0
    current_location = points[0]

    for i in range(1, len(points)):
        segment_distance = geodesic(current_location, points[i]).km
        distance_traveled += segment_distance

        if distance_traveled > max_range - buffer_range:
            charging_stations = find_charging_stations(current_location[0], current_location[1], charging_api_key)

            if charging_stations:
                closest_station = min(charging_stations, key=lambda s: geodesic((s["AddressInfo"]["Latitude"], s["AddressInfo"]["Longitude"]), current_location).km)
                stops.append({
                    "location": closest_station["AddressInfo"]["Title"],
                    "address": closest_station["AddressInfo"]["AddressLine1"],
                    "coords": (closest_station["AddressInfo"]["Latitude"], closest_station["AddressInfo"]["Longitude"])
                })

                current_location = (closest_station["AddressInfo"]["Latitude"], closest_station["AddressInfo"]["Longitude"])
                distance_traveled = 0
            else:
                raise Exception("No charging stations found near the route")

        current_location = points[i]

    return stops

def main():
    # Updated origin and destination coordinates
    origin = "-37.8136,144.9631"  # Melbourne, VIC, Australia
    destination = "-38.1499,144.3617"  # Geelong, VIC, Australia

    google_maps_api_key = "YOUR_GOOGLE_MAPS_API_KEY"
    charging_api_key = "YOUR_OPENCHARGEMAP_API_KEY"

    max_range = 300  # in km
    starting_range = 100  # in km
    buffer_range = 50  # in km

    route_points = get_route(origin, destination, google_maps_api_key)
    stops = find_stops(route_points, max_range, starting_range, buffer_range, charging_api_key)

    print("Charging Stops:")
    for stop in stops:
        print(f"Location: {stop['location']}, Address: {stop['address']}, Coordinates: {stop['coords']}")

if __name__ == "__main__":
    main()
